# Gmail API Usage Examples

This notebook demonstrates how to:
- Read email subjects
- Get email receive date and time
- Check read/unread status
- List attachments
- Download attachments

## Setting up credentials.json

Before running this notebook, you need to set up your Google Cloud Project and obtain the credentials:

1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select an existing one
3. Enable the Gmail API:
   - In the left sidebar, click on "APIs & Services" > "Library"
   - Search for "Gmail API"
   - Click on it and click "Enable"

4. Create credentials:
   - Go to "APIs & Services" > "Credentials"
   - Click "Create Credentials" > "OAuth client ID"
   - If prompted, configure the OAuth consent screen:
     - Choose "External" user type
     - Fill in the required fields (app name, user support email, developer contact)
     - For scopes, add the Gmail API scope you need (../auth/gmail.readonly)
     - Add your email as a test user
   
5. Create the OAuth client ID:
   - Choose "Desktop app" as application type
   - Give it a name (e.g., "Gmail API Client")
   - Click "Create"

6. Download the credentials:
   - After creation, click the download button (JSON)
   - Save the file as `credentials.json` in the same directory as this notebook

The first time you run the code, it will open a web browser for authentication. After authenticating, the token will be saved locally in `token.pickle` for future use.

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import base64
import pickle
import os.path
from datetime import datetime
import email

In [2]:
# If modifying these scopes, delete the file token.pickle
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def get_gmail_service():
    """Get authenticated Gmail API service instance."""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return build('gmail', 'v1', credentials=creds)

# Initialize the Gmail API service
service = get_gmail_service()

## 1. List Recent Emails with Details
Get the last 5 emails with their subjects, dates, and read/unread status

In [ ]:
def get_email_details(service, message_id):
    """Get details of a specific email."""
    msg = service.users().messages().get(userId='me', id=message_id).execute()
    
    # Get email headers
    headers = msg['payload']['headers']
    subject = next((header['value'] for header in headers if header['name'] == 'Subject'), 'No Subject')
    date = next((header['value'] for header in headers if header['name'] == 'Date'), 'No Date')
    
    # Convert date string to datetime object
    try:
        date_obj = email.utils.parsedate_to_datetime(date)
        formatted_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
    except:
        formatted_date = date
    
    # Check read/unread status
    is_unread = 'UNREAD' in msg['labelIds']
    
    return {
        'subject': subject,
        'date': formatted_date,
        'status': 'Unread' if is_unread else 'Read',
        'id': message_id
    }

# Get list of messages
results = service.users().messages().list(userId='me', maxResults=5, labelIds=['INBOX']).execute()
messages = results.get('messages', [])

# Get details for each message
for message in messages:
    details = get_email_details(service, message['id'])
    print(f"Subject: {details['subject']}")
    print(f"Date: {details['date']}")
    print(f"Status: {details['status']}")
    print('-' * 50)

## 2. List and Download Attachments
Show how to list attachments and download them

In [ ]:
def get_attachments(service, message_id):
    """Get list of attachments for a specific email."""
    message = service.users().messages().get(userId='me', id=message_id).execute()
    parts = message.get('payload', {}).get('parts', [])
    attachments = []
    
    for part in parts:
        if part.get('filename') and part.get('body', {}).get('attachmentId'):
            attachments.append({
                'id': part['body']['attachmentId'],
                'filename': part['filename'],
                'mimeType': part['mimeType']
            })
    
    return attachments

def download_attachment(service, message_id, attachment_id, filename):
    """Download a specific attachment."""
    attachment = service.users().messages().attachments().get(
        userId='me', messageId=message_id, id=attachment_id
    ).execute()
    
    file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
    
    # Create 'downloads' directory if it doesn't exist
    if not os.path.exists('downloads'):
        os.makedirs('downloads')
    
    filepath = os.path.join('downloads', filename)
    with open(filepath, 'wb') as f:
        f.write(file_data)
    
    return filepath

In [ ]:
# Example: List attachments for the first email that has attachments
for message in messages:
    attachments = get_attachments(service, message['id'])
    if attachments:
        details = get_email_details(service, message['id'])
        print(f"Email Subject: {details['subject']}")
        print("Attachments:")
        for attachment in attachments:
            print(f"- {attachment['filename']} ({attachment['mimeType']})")
            
            # Download the attachment
            filepath = download_attachment(
                service, 
                message['id'], 
                attachment['id'], 
                attachment['filename']
            )
            print(f"  Downloaded to: {filepath}")
        print('-' * 50)
        break  # Stop after finding first email with attachments